In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-12 03:11:29--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.4’

postgresql-42.2.16. 100%[===================>] 979.38K  5.57MB/s    in 0.2s    

2021-08-12 03:11:29 (5.57 MB/s) - ‘postgresql-42.2.16.jar.4’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url_us_total ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/us_total.csv"
spark.sparkContext.addFile(url_us_total)
us_total_df = spark.read.csv(SparkFiles.get("us_total.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
us_total_df.show()

+----------+---------+-------------+----------+----------+--------------------+---------------+
|      date|size_rank|  region_name|list_price|sale_price|list_divided_by_sale|list_minus_sale|
+----------+---------+-------------+----------+----------+--------------------+---------------+
|2017-11-04|        0|United States|  275448.0|  235099.0|   85.35150010165258|        40349.0|
|2017-11-11|        0|United States|  275448.0|  234750.0|   85.22479742092881|        40698.0|
|2017-11-18|        0|United States|  275448.0|  235750.0|    85.5878423513694|        39698.0|
|2017-11-25|        0|United States|  274975.0|  235916.0|   85.79543594872261|        39059.0|
|2017-12-02|        0|United States|  274825.0|  237166.0|   86.29709815337033|        37659.0|
|2017-12-09|        0|United States|  274075.0|  238591.0|    87.0531788743957|        35484.0|
|2017-12-16|        0|United States|  272825.0|  238841.0|   87.54366352057178|        33984.0|
|2017-12-23|        0|United States|  27

In [6]:
url_ca_state ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/ca_state.csv"
spark.sparkContext.addFile(url_ca_state)
ca_state_df = spark.read.csv(SparkFiles.get("ca_state.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
ca_state_df.show()

+----------+---------+-----------+----------+----------+--------------------+------------------+
|      date|size_rank|region_name|list_price|sale_price|list_divided_by_sale|   list_minus_sale|
+----------+---------+-----------+----------+----------+--------------------+------------------+
|2017-11-04|     36.4| California|  614123.7|  508281.3|   82.76529630756801|105842.39999999997|
|2017-11-11|     36.4| California|  613946.7|  514450.0|   83.79391891836865| 99496.69999999995|
|2017-11-18|     36.4| California|  614898.4|  516943.9|   84.06980730475149|           97954.5|
|2017-11-25|     36.4| California|  612118.7|  517912.7|    84.6098477305137| 94205.99999999994|
|2017-12-02|     36.4| California|  608753.3|  517612.8|   85.02833578068487| 91140.50000000006|
|2017-12-09|     36.4| California|  608399.3|  517747.5|    85.0999499835059| 90651.80000000005|
|2017-12-16|     36.4| California|  608267.7|  511941.2|   84.16379827500295| 96326.49999999994|
|2017-12-23|     36.4| Califor

In [7]:
url_ca_city ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/california_city.csv"
spark.sparkContext.addFile(url_ca_city)
ca_city_df = spark.read.csv(SparkFiles.get("california_city.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
ca_city_df.show()

+----------+---------+--------------------+----------+----------+--------------------+---------------+
|      date|size_rank|         region_name|list_price|sale_price|list_divided_by_sale|list_minus_sale|
+----------+---------+--------------------+----------+----------+--------------------+---------------+
|2017-11-04|       17|       San Diego, CA|  728005.0|  541438.0|   74.37284084587331|       186567.0|
|2017-11-04|       11|   San Francisco, CA|  867930.0|  781063.0|   89.99147396679456|        86867.0|
|2017-11-04|       13|       Riverside, CA|  388757.0|  342999.0|   88.22966531792353|        45758.0|
|2017-11-04|        2|Los Angeles-Long ...|  812252.0|  611250.0|   75.25373898740784|       201002.0|
|2017-11-04|       77|        Stockton, CA|  408007.0|  357875.0|   87.71295590516829|        50132.0|
|2017-11-04|       66|         Ventura, CA|  779777.0|  583688.0|   74.85319520837368|       196089.0|
|2017-11-04|       25|      Sacramento, CA|  466317.0|  392750.0|   84.22

In [8]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ucb-data-group-project-zillow.csaw135fqqkl.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [10]:
# Write DataFrame to zillow_final table in RDS
us_total_df.write.jdbc(url=jdbc_url, table='zillow_final', mode=mode, properties=config)

In [11]:
# Write DataFrame to zillow_final table in RDS
ca_state_df.write.jdbc(url=jdbc_url, table='zillow_final', mode=mode, properties=config)

In [12]:
# Write DataFrame to zillow_final table in RDS
ca_city_df.write.jdbc(url=jdbc_url, table='zillow_final', mode=mode, properties=config)